In [9]:
import tensorflow as tf 
import cv2 
import os 

In [10]:
rootDir = 'A:\Lip Reading\Potential Datasets\BBC LRS2\\allFiles'
vocab = [x for x in "ABCDEFGHIJKLMNOPQRSTUVWXYZ'?!123456789 "]

def getFrameCount(path) -> int: 
    cap = cv2.VideoCapture(path)
    frameCount = cap.get(cv2.CAP_PROP_FRAME_COUNT)
    cap.release()
    return frameCount
def getCharCount(path) -> int: 
    return len(loadText(path))
def loadText(path): 
    # open and parse the file 
    with open(path, 'r') as file: lines = file.readlines()
    file.close()
    
    # return the number equivalent of each of the characters of the word 
    tokens = []
    words = lines[0].split()
    del words[0]
    for word in words: tokens = [*tokens,' ', word]
    
    return charToNum(tf.reshape(tf.strings.unicode_split(tokens, input_encoding='UTF-8'), (-1)))[1:]   
charToNum = tf.keras.layers.StringLookup(vocabulary=vocab, oov_token="")


In [11]:
# finding the max number of frames & chars in a certain video 
frameCt = []
charCt = []
vidList = []
textList = []
fileList = [vidList, textList]
# iterating through each file in the root folder
for subdir, dirs, files in os.walk(rootDir):
    for file in files:
        if file.split('.')[1] == "mp4": 
            frameCt.append(getFrameCount(os.path.join(subdir, file)))
            vidList.append(os.path.join(subdir, file))
        if file.split('.')[1] == "txt": 
            charCt.append(getCharCount(os.path.join(subdir, file)))
            textList.append(os.path.join(subdir, file))
        
            
print("frameCt", max(frameCt), ", index:", frameCt.index(max(frameCt)))
print("longest video:", vidList[frameCt.index(max(frameCt))])
print("charCt", max(charCt), ", index:", charCt.index(max(charCt)))
print("longest text:", textList[charCt.index(max(charCt))])


frameCt 154.0 , index: 4207
longest video: A:\Lip Reading\Potential Datasets\BBC LRS2\allFiles\5682340946807479970_00031.mp4
charCt 99 , index: 510
longest text: A:\Lip Reading\Potential Datasets\BBC LRS2\allFiles\5540854416155721607_00004.txt
